# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), mlb (basketball, in [assets/mlb.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

As this assignment utilizes global variables in the skeleton code, to avoid having errors in your code you can either:

1. You can place all of your code within the function definitions for all of the questions (other than import statements).
2. You can create copies of all the global variables with the copy() method and proceed as usual.

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [140]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nhl_df = pd.read_csv("assets/nhl.csv")
nhl_df = nhl_df[
    (~nhl_df["team"].str.contains("Division")) & (nhl_df["year"] == 2018)
].reset_index(drop=True)
nhl_df["team"] = nhl_df["team"].str.replace("*", "")
nhl_df = nhl_df.assign(
    metropolitan_area=pd.Series(
        [
            "Tampa Bay Area",
            "Boston",
            "Toronto",
            "Miami–Fort Lauderdale",
            "Detroit",
            "Montreal",
            "Ottawa",
            "Buffalo",
            "Washington, D.C.",
            "Pittsburgh",
            "Philadelphia",
            "Columbus",
            "New York City",
            "Raleigh",
            "New York City",
            "New York City",
            "Nashville",
            "Winnipeg",
            "Minneapolis–Saint Paul",
            "Denver",
            "St. Louis",
            "Dallas–Fort Worth",
            "Chicago",
            "Las Vegas",
            "Los Angeles",
            "San Francisco Bay Area",
            "Los Angeles",
            "Calgary",
            "Edmonton",
            "Vancouver",
            "Phoenix",
        ]
    )
)

nhl_df["win loss ratio"] = nhl_df["W"].astype("int") / (
    nhl_df["W"].astype("int") + nhl_df["L"].astype("int")
)
cities = pd.read_html("assets/wikipedia_data.html")[1]
cities = cities.iloc[:-1, [0, 3, 5, 6, 7, 8]]

nhl_df = pd.merge(
    nhl_df,
    cities[["Metropolitan area", "Population (2016 est.)[8]"]],
    how="left",
    left_on="metropolitan_area",
    right_on="Metropolitan area",
)

nhl_df = nhl_df.rename(columns={"Population (2016 est.)[8]": "Population"})
nhl_df_teams = (
    nhl_df[["metropolitan_area", "win loss ratio"]]
    .groupby(["metropolitan_area"])
    .mean()
)
metro_areas = pd.merge(
    nhl_df_teams,
    cities,
    how="left",
    left_on="metropolitan_area",
    right_on="Metropolitan area",
).rename(columns={"Population (2016 est.)[8]": "Population"})


def nhl_correlation():

    population_by_region = (
        metro_areas["Population"].astype(int).to_list()
    )  # pass in metropolitan area population from cities
    win_loss_by_region = nhl_df_teams[
        "win loss ratio"
    ].to_list()  # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(
        win_loss_by_region
    ), "Q1: Your lists must be the same length"
    assert (
        len(population_by_region) == 28
    ), "Q1: There should be 28 teams being analysed for NHL"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

In [142]:
nhl_correlation()

0.012486162921209902

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [175]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nba_df = pd.read_csv("assets/nba.csv")
nba_df = nba_df[nba_df["year"] == 2018].reset_index(drop=True)
nba_df["team"] = nba_df["team"].str.replace("\*?\s\(\d+\)", "", regex=True)
nba_df = nba_df.assign(
    metropolitan_area=pd.Series(
        [
            "Toronto",
            "Boston",
            "Philadelphia",
            "Cleveland",
            "Indianapolis",
            "Miami–Fort Lauderdale",
            "Milwaukee",
            "Washington, D.C.",
            "Detroit",
            "Charlotte",
            "New York City",
            "New York City",
            "Chicago",
            "Orlando",
            "Atlanta",
            "Houston",
            "San Francisco Bay Area",
            "Portland",
            "Oklahoma City",
            "Salt Lake City",
            "New Orleans",
            "San Antonio",
            "Minneapolis–Saint Paul",
            "Denver",
            "Los Angeles",
            "Los Angeles",
            "Sacramento",
            "Dallas–Fort Worth",
            "Memphis",
            "Phoenix",
        ]
    )
)
nba_df["win loss ratio"] = nba_df["W"].astype("int") / (
    nba_df["W"].astype("int") + nba_df["L"].astype("int")
)
cities = pd.read_html("assets/wikipedia_data.html")[1]
cities = cities.iloc[:-1, [0, 3, 5, 6, 7, 8]]

nba_df = pd.merge(
    nba_df,
    cities[["Metropolitan area", "Population (2016 est.)[8]"]],
    how="left",
    left_on="metropolitan_area",
    right_on="Metropolitan area",
)

nba_df = nba_df.rename(columns={"Population (2016 est.)[8]": "Population"})
nba_df_teams = (
    nba_df[["metropolitan_area", "win loss ratio"]]
    .groupby(["metropolitan_area"])
    .mean()
)
metro_areas = pd.merge(
    nba_df_teams,
    cities,
    how="left",
    left_on="metropolitan_area",
    right_on="Metropolitan area",
).rename(columns={"Population (2016 est.)[8]": "Population"})


def nba_correlation():
    population_by_region = (
        metro_areas["Population"].astype(int).to_list()
    )  # pass in metropolitan area population from cities
    win_loss_by_region = nba_df_teams[
        "win loss ratio"
    ].to_list()  # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(
        win_loss_by_region
    ), "Q2: Your lists must be the same length"
    assert (
        len(population_by_region) == 28
    ), "Q2: There should be 28 teams being analysed for NBA"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

In [176]:
nba_correlation()

-0.17657160252844617

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [204]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df = pd.read_csv("assets/mlb.csv")
mlb_df = mlb_df[mlb_df["year"] == 2018].reset_index(drop=True)
mlb_df = mlb_df.assign(
    metropolitan_area=pd.Series(
        [
            "Boston",
            "New York City",
            "Tampa Bay Area",
            "Toronto",
            "Baltimore",
            "Cleveland",
            "Minneapolis–Saint Paul",
            "Detroit",
            "Chicago",
            "Kansas City",
            "Houston",
            "San Francisco Bay Area",
            "Seattle",
            "Los Angeles",
            "Dallas–Fort Worth",
            "Atlanta",
            "Washington, D.C.",
            "Philadelphia",
            "New York City",
            "Miami–Fort Lauderdale",
            "Milwaukee",
            "Chicago",
            "St. Louis",
            "Pittsburgh",
            "Cincinnati",
            "Los Angeles",
            "Denver",
            "Phoenix",
            "San Francisco Bay Area",
            "San Diego",
        ]
    )
)
mlb_df["win loss ratio"] = mlb_df["W"].astype("int") / (
    mlb_df["W"].astype("int") + mlb_df["L"].astype("int")
)
cities = pd.read_html("assets/wikipedia_data.html")[1]
cities = cities.iloc[:-1, [0, 3, 5, 6, 7, 8]]

mlb_df = pd.merge(
    mlb_df,
    cities[["Metropolitan area", "Population (2016 est.)[8]"]],
    how="left",
    left_on="metropolitan_area",
    right_on="Metropolitan area",
)

mlb_df = mlb_df.rename(columns={"Population (2016 est.)[8]": "Population"})
mlb_df_teams = (
    mlb_df[["metropolitan_area", "win loss ratio"]]
    .groupby(["metropolitan_area"])
    .mean()
)
metro_areas = pd.merge(
    mlb_df_teams,
    cities,
    how="left",
    left_on="metropolitan_area",
    right_on="Metropolitan area",
).rename(columns={"Population (2016 est.)[8]": "Population"})


def mlb_correlation():
    population_by_region = (
        metro_areas["Population"].astype(int).to_list()
    )  # pass in metropolitan area population from cities
    win_loss_by_region = mlb_df_teams[
        "win loss ratio"
    ].to_list()  # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(
        win_loss_by_region
    ), "Q3: Your lists must be the same length"
    assert (
        len(population_by_region) == 26
    ), "Q3: There should be 26 teams being analysed for MLB"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

In [205]:
mlb_correlation()

0.15027698302669307

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [236]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nfl_df = pd.read_csv("assets/nfl.csv")

nfl_df = nfl_df[
    ~((nfl_df["team"].str.contains("AFC")) | (nfl_df["team"].str.contains("NFC")))
    & (nfl_df["year"] == 2018)
].reset_index(drop=True)
nfl_df["team"] = nfl_df["team"].str.replace("\*|\+", "", regex=True)
nfl_df = nfl_df.assign(
    metropolitan_area=pd.Series(
        [
            "Boston",
            "Miami–Fort Lauderdale",
            "Buffalo",
            "New York City",
            "Baltimore",
            "Pittsburgh",
            "Cleveland",
            "Cincinnati",
            "Houston",
            "Indianapolis",
            "Nashville",
            "Jacksonville",
            "Kansas City",
            "Los Angeles",
            "Denver",
            "San Francisco Bay Area",
            "Dallas–Fort Worth",
            "Philadelphia",
            "Washington, D.C.",
            "New York City",
            "Chicago",
            "Minneapolis–Saint Paul",
            "Green Bay",
            "Detroit",
            "New Orleans",
            "Charlotte",
            "Atlanta",
            "Tampa Bay Area",
            "Los Angeles",
            "Seattle",
            "San Francisco Bay Area",
            "Phoenix",
        ]
    )
)
nfl_df["win loss ratio"] = nfl_df["W"].astype("int") / (
    nfl_df["W"].astype("int") + nfl_df["L"].astype("int")
)
cities = pd.read_html("assets/wikipedia_data.html")[1]
cities = cities.iloc[:-1, [0, 3, 5, 6, 7, 8]]

nfl_df = pd.merge(
    nfl_df,
    cities[["Metropolitan area", "Population (2016 est.)[8]"]],
    how="left",
    left_on="metropolitan_area",
    right_on="Metropolitan area",
)

nfl_df = nfl_df.rename(columns={"Population (2016 est.)[8]": "Population"})
nfl_df_teams = (
    nfl_df[["metropolitan_area", "win loss ratio"]]
    .groupby(["metropolitan_area"])
    .mean()
)
metro_areas = pd.merge(
    nfl_df_teams,
    cities,
    how="left",
    left_on="metropolitan_area",
    right_on="Metropolitan area",
).rename(columns={"Population (2016 est.)[8]": "Population"})


def nfl_correlation():
    population_by_region = (
        metro_areas["Population"].astype(int).to_list()
    )  # pass in metropolitan area population from cities
    win_loss_by_region = nfl_df_teams[
        "win loss ratio"
    ].to_list()  # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(
        win_loss_by_region
    ), "Q4: Your lists must be the same length"
    assert (
        len(population_by_region) == 29
    ), "Q4: There should be 29 teams being analysed for NFL"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

In [237]:
nfl_correlation()

0.004922112149349462

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
nhl_df=pd.read_csv("assets/nhl.csv")
nba_df=pd.read_csv("assets/nba.csv")
nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def sports_team_performance():
    # YOUR CODE HERE
    raise NotImplementedError()
    
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values